### Image Classifier using `CNN` in pytorch
Right now we have `240` images of each class and we have `10` classes which are: 
```
{"camera": 0, "cellphone": 1, "charger": 2, "desktop": 3, "drone": 4, "headphone": 5, "laptop": 6, "mouse": 7, "remote": 8, "television": 9}
```

### Imports

In [1]:
import os, cv2, json
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:
class Electronics:
    CLASS_NAMES = "class_names.json"
    ROOT = 'electronics'
    
    CAMERA = "camera"
    CELLPHONE = 'cellphone'
    CHARGER = 'charger'
    DESKTOP = 'desktop'
    DRONE = 'drone'
    HEADPHONE = 'headphone'
    LAPTOP = 'laptop'
    MOUSE = 'mouse'
    TELEVISION = 'television'
    REMOTE = 'remote'
    
    GRAY = 'gray'
    BGR = 'bgr'
    RGB = 'rgb'


In [3]:
with open(os.path.join(Electronics.ROOT, Electronics.CLASS_NAMES), 'r') as f:
    class_names = dict(json.load(f))

In [4]:
class_names.keys()

dict_keys(['camera', 'cellphone', 'charger', 'desktop', 'drone', 'headphone', 'laptop', 'mouse', 'remote', 'television'])

In [5]:
all_images = []
all_labels = []
for category in list(class_names.keys()): 
    all_images_path =  os.listdir(os.path.join(os.path.join(Electronics.ROOT, category), Electronics.BGR))
    for image_path in all_images_path:
        abs_image_path = os.path.join(os.path.join(os.path.join(Electronics.ROOT, 
                                                                category), Electronics.BGR), image_path)
        image = cv2.imread(abs_image_path, cv2.IMREAD_UNCHANGED)
        all_images.append(image)
        all_labels.append(class_names[category])
print("Done")

Done


In [6]:
all_images_array = np.array(all_images)
all_labels_array = np.array(all_labels)

In [7]:
all_labels_array.shape, all_images_array.shape

((2400,), (2400, 100, 100, 3))

In [8]:
one_hot_labels = np.array([np.eye(10)[i] for i in all_labels_array])


> Creating an `Electronic` Dataset

In [24]:
class Electronic(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.x = all_images_array
        self.y = all_labels_array
        
        self.len = one_hot_labels.shape[0]
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        samples = self.x[index], self.y[index]
        if self.transform:
            samples = self.transform(samples)
        return samples
    
electronic = Electronic()

## Transformers

> `ToTensor` Transformer - This converts features and labels to pytorch `tensors`

In [25]:
class ToTensor:
    def __call__(self, sample):
        x, y = sample
        return torch.Tensor(x.astype('float32')), torch.from_numpy(np.array(y.astype('float32'))).long()

> `Normalize` Transformer - This normalize features for each pixel to be between `0` and `1`

In [26]:
class Normalize:
    def __call__(self, sample):
        x, y = sample
        return x/255, np.array(y) 

In [27]:
electronic = Electronic(transform = transforms.Compose([
    Normalize(),
    ToTensor()
]))

In [28]:
electronic[0]

(tensor([[[0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          ...,
          [0.5373, 0.5373, 0.5373],
          [0.5333, 0.5333, 0.5333],
          [0.5765, 0.5765, 0.5765]],
 
         [[0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          ...,
          [0.7647, 0.7647, 0.7647],
          [0.7059, 0.7059, 0.7059],
          [0.6431, 0.6431, 0.6431]],
 
         [[0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          ...,
          [0.9176, 0.9176, 0.9176],
          [0.7451, 0.7451, 0.7451],
          [0.5882, 0.5882, 0.5882]],
 
         ...,
 
         [[0.2667, 0.2667, 0.2667],
          [0.2588, 0.2588, 0.2588],
          [0.2588, 0.2588, 0.2588],
          ...,
          [0.2392, 0.2392, 0.2392],
          [0.2157, 0.2157, 0.2157],
          [0.2275, 0.2275, 0.2275]],
 
         [[0.3137, 0.3137, 0.3137],
          [0

> Spliting data into `train` and `testing`

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
train, test = train_test_split(electronic, random_state=42, test_size=.3)

> Data ``Loaders``

In [31]:
train_set = DataLoader(train, 
                       batch_size=32,
                       shuffle=True
                      )
test_set = DataLoader(train,
                      batch_size=32,
                      shuffle=False
                     )

### Creating a `Image Classifier` model `CNN`

In [32]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 32, kernel_size=(3, 3))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels= 256, kernel_size=(2, 2))
        self.conv3 = nn.Conv2d(in_channels=256, out_channels= 64, kernel_size=(2, 2))
        self.conv4 = nn.Conv2d(in_channels=64, out_channels= 16, kernel_size=(2, 2))
        
        self._to_linear = None
        self.x = torch.randn(3, 100, 100).view(-1, 3, 100, 100)
        self.conv(self.x)
        
        self.fc1 = nn.Linear(self._to_linear, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
                               
    def conv(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv4(x)), (2, 2))
        
        if self._to_linear is None:
            self._to_linear = x.shape[1] * x.shape[2] * x.shape[3]
        return x
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    
net = Net()
net

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 256, kernel_size=(2, 2), stride=(1, 1))
  (conv3): Conv2d(256, 64, kernel_size=(2, 2), stride=(1, 1))
  (conv4): Conv2d(64, 16, kernel_size=(2, 2), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

In [39]:
def accuracy(y_pred, y_true):
    print(y_pred, y_true)
    with torch.no_grad():
        total = 0
        correct = 0
        for i in range(len(y_true)):
            if y_pred[i] == y_true[i]:
                correct +=1
            total +=1
        return correct/total

In [42]:
EPOCHS = 20

for epoch in range(EPOCHS):
    y_true = None
    y_pred = None
    for X, y in train_set:
        optimizer.zero_grad()
        # forward pass
        outputs = net(X.view(-1, 3, 100, 100))
#         print(outputs, y)
        y_true = y
        y_pred = torch.argmax(outputs.detach(), dim=1)
        # loss 
        loss = criterion(outputs, y)
        #backward pass
        loss.backward()
        # update weights
        optimizer.step()
    accu = accuracy(y_pred, y_true)
    print(f'Epoch {epoch+1}/{EPOCHS}')
    print(f"loss: {loss.item():.3f}, accuracy: {accu:.3f}")

tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]) tensor([1, 4, 0, 0, 9, 9, 0, 3, 7, 3, 4, 7, 4, 0, 1, 2])
Epoch 1/20
loss: 2.309, accuracy: 0.000
tensor([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]) tensor([9, 2, 8, 6, 3, 6, 4, 6, 3, 4, 1, 9, 6, 3, 3, 9])
Epoch 2/20
loss: 2.282, accuracy: 0.188
tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]) tensor([2, 2, 9, 2, 0, 4, 0, 5, 0, 0, 0, 8, 5, 6, 0, 4])
Epoch 3/20
loss: 2.315, accuracy: 0.062
tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]) tensor([5, 2, 4, 7, 3, 0, 6, 1, 9, 9, 0, 1, 5, 6, 3, 1])
Epoch 4/20
loss: 2.311, accuracy: 0.125
tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]) tensor([8, 4, 6, 2, 9, 5, 4, 8, 0, 6, 5, 0, 3, 0, 6, 0])
Epoch 5/20
loss: 2.292, accuracy: 0.188
tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]) tensor([0, 2, 9, 4, 8, 1, 1, 4, 0, 4, 8, 3, 7, 9, 8, 8])
Epoch 6/20
loss: 2.313, accuracy: 0.000
tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]) tensor([6, 7, 8, 6,

> **Conclusion** - The ``model`` is guessing, it's not learning anything from **features**.